In [ ]:
import sys
import os
current = os.path.dirname(os.path.realpath('./'))
sys.path.append(current)
from utils.ld_tools import *
from celluloid import Camera # getting the camera

In [ ]:
def reconstruct_path(pred,idx1,idx2):
    idx = idx2
    path = [idx2]
    while idx != idx1:
        path += [pred[idx1,idx]]
        idx = pred[idx1,idx]
    path += [idx1]
    return path

In [ ]:
from sklearn import datasets
points, y = datasets.make_moons(n_samples=2000,noise=0.1,random_state=1)
points = points[np.where(y==0)]
minX1, maxX1, minX2, maxX2 = np.min(points[:,0])-0.5, np.max(points[:,0])+0.5, np.min(points[:,1])-0.5, np.max(points[:,1])+0.5
# create one-dimensional arrays for x and y
x1 = np.linspace(minX1, maxX1, 30)
x2 = np.linspace(minX2, maxX2, 30)
# create the mesh based on these arrays
X1, X2 = np.meshgrid(x1, x2)
X1 = X1.reshape(-1,)
X2 = X2.reshape(-1,)
points_ext = np.zeros((len(X1),2))
points_ext[:,0] = X1
points_ext[:,1] = X2
arg = np.argsort(points[:,0])
idx1 = arg[int(0.1*points.shape[0])]
idx2 = arg[int(0.9*points.shape[0])]

In [ ]:
## spiral
N2 = 1100
theta = np.sqrt(np.random.rand(N2))*3*math.pi
r_a = 2*theta + math.pi
data_a = np.array([np.cos(theta)*r_a, np.sin(theta)*r_a]).T
points2 = data_a + np.random.randn(N2,2)
minX1, maxX1, minX2, maxX2 = np.min(points2[:,0])-5, np.max(points2[:,0])+5, np.min(points2[:,1])-5, np.max(points2[:,1])+5
# create one-dimensional arrays for x and y
x1 = np.linspace(minX1, maxX1, 30)
x2 = np.linspace(minX2, maxX2, 30)
# create the mesh based on these arrays
X12, X22 = np.meshgrid(x1, x2)
X12 = X12.reshape(-1,)
X22 = X22.reshape(-1,)
points_ext2 = np.zeros((len(X12),2))
points_ext2[:,0] = X12
points_ext2[:,1] = X22

arg2 = np.argsort(theta)
idx12 = arg2[int(0.06*points2.shape[0])]
idx22 = arg2[int(0.95*points2.shape[0])]

In [ ]:
plt.rcParams['figure.figsize'] = [10,10]
fig, ax = plt.subplots(2,2)
camera = Camera(fig)

for alpha in [1.,1.05,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.5,3.0,4.0,5.0,6.0,7.0]:
#for alpha in [1.]:
    print(alpha)
    dist_mat = distance_matrix(points,points)
    distances, pred = shortest_path(
                csgraph=np.matrix(np.power(dist_mat, alpha)),
                method='FW',
                directed=False,
                return_predecessors=True
            )

    path = reconstruct_path(pred,idx1,idx2)
    
    
    dist_mat = distance_matrix(points,points)
    fermat_dist_mat = fermat_function(dist_mat,alpha=alpha)
    lens_depth_ext_fermat = []
    for i in range(points_ext.shape[0]):
        dist_array = distance_matrix(points,points_ext[i].reshape((1,-1)))
        dist_ext = fermat_function_ext(fermat_dist_mat, dist_array,alpha)
        lens_depth_ext_fermat += [lens_depth_ext_function(fermat_dist_mat, dist_ext)]
  
    #ax[0].scatter(points[:,0],points[:,1],color='green')
    ax[0,0].plot(points[:,0], points[:,1], 'ko', ms=3)
    ax[0,0].scatter(points[[idx1,idx2],0],points[[idx1,idx2],1],color='red')
    #plt.grid()
    ax[0,0].plot(points[path,0],points[path,1],color='red',linewidth=3.5)
    ax[0,0].set(xlim=(np.min(points[:,0])-0.5, np.max(points[:,0])+0.5), ylim=(np.min(points[:,1])-0.5, np.max(points[:,1])+0.5))
    ax[0,0].set_xlabel('Sample Fermat Path')
#     ax[0].gca().axes.get_yaxis().set_visible(False)
#     ax[0].gca().axes.get_xaxis().set_visible(False)
    
    ax[0,1].tricontour(points_ext[:,0], points_ext[:,1], lens_depth_ext_fermat, levels=20, linewidths=0.5, colors='k')
    cntr = ax[0,1].tricontourf(points_ext[:,0], points_ext[:,1], lens_depth_ext_fermat, levels=20, cmap="RdBu_r")
    #fig.colorbar(cntr, ax=ax)
    #ax.plot(X_grid[:,0], X_grid[:,1], 'ko', ms=1, color="white")
    ax[0,1].plot(points[:,0], points[:,1], 'ko', ms=3)
    ax[0,1].set(xlim=(np.min(points[:,0])-0.5, np.max(points[:,0])+0.5), ylim=(np.min(points[:,1])-0.5, np.max(points[:,1])+0.5))
    ax[0,1].set_xlabel('LD using modified SFD')
    
    ax[0,0].text(0.9, 1.03, f'Value of $\\alpha$: {alpha}', fontsize='xx-large', transform=ax[0,0].transAxes) 
    
    
    dist_mat = distance_matrix(points2,points2)
    distances, pred = shortest_path(
                csgraph=np.matrix(np.power(dist_mat, alpha)),
                method='FW',
                directed=False,
                return_predecessors=True
            )

    path = reconstruct_path(pred,idx12,idx22)
    
    
    dist_mat = distance_matrix(points2,points2)
    fermat_dist_mat = fermat_function(dist_mat,alpha=alpha)
    lens_depth_ext_fermat = []
    for i in range(points_ext.shape[0]):
        dist_array = distance_matrix(points2,points_ext2[i].reshape((1,-1)))
        dist_ext = fermat_function_ext(fermat_dist_mat, dist_array,alpha)
        lens_depth_ext_fermat += [lens_depth_ext_function(fermat_dist_mat, dist_ext)]
  
    #ax[0].scatter(points[:,0],points[:,1],color='green')
    ax[1,0].plot(points2[:,0], points2[:,1], 'ko', ms=3)
    ax[1,0].scatter(points2[[idx12,idx22],0],points2[[idx12,idx22],1],color='red')
    #plt.grid()
    ax[1,0].plot(points2[path,0],points2[path,1],color='red',linewidth=3.5)
    ax[1,0].set(xlim=(np.min(points2[:,0])-5, np.max(points2[:,0])+5), ylim=(np.min(points2[:,1])-5, np.max(points2[:,1])+5))
    ax[1,0].set_xlabel('Sample Fermat Path')
#     ax[0].gca().axes.get_yaxis().set_visible(False)
#     ax[0].gca().axes.get_xaxis().set_visible(False)
    
    ax[1,1].tricontour(points_ext2[:,0], points_ext2[:,1], lens_depth_ext_fermat, levels=20, linewidths=0.5, colors='k')
    cntr = ax[1,1].tricontourf(points_ext2[:,0], points_ext2[:,1], lens_depth_ext_fermat, levels=20, cmap="RdBu_r")
    ax[1,1].plot(points2[:,0], points2[:,1], 'ko', ms=3)
    ax[1,1].set(xlim=(np.min(points2[:,0])-5, np.max(points2[:,0])+5), ylim=(np.min(points2[:,1])-5, np.max(points2[:,1])+5))
    ax[1,1].set_xlabel('LD using modified SFD')
    
    plt.show()
    camera.snap()

animation = camera.animate()
animation.save('animations.gif', writer='PillowWriter', fps=3)
print("Done!!!!")